## Golang Refactor. Given a TS code, refactor it to Golang.

In [67]:
# imports

import os
import io
import sys
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [68]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')


In [69]:
openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "o3-mini"
CLAUDE_MODEL = "claude-3-7-sonnet-latest"

In [70]:
system_message = "You are an assistant that reimplements Typescript code in performance GoLang. "
system_message += "Respond only with GoLang code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The GoLang response needs to produce an identical output in the fastest possible time."

In [71]:
def user_prompt_for(ts_code):
    user_prompt = "Rewrite this Typescript code in GoLang with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with GoLang code; do not explain your work other than a few comments. "
    user_prompt += "Implement tests for all functions."
    user_prompt += "Implement all functions and classes. Do not leave any functionality out.\n\n"
    user_prompt += ts_code
    return user_prompt

In [72]:
def messages_for(ts_code):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(ts_code)}
    ]

In [73]:
def write_output(golang_code):
    code = golang_code.replace("```go","").replace("```","")
    with open("optimized.go", "w") as f:
        f.write(code)

In [74]:
def optimize_gpt(golang_code):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(golang_code), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [75]:
def optimize_claude(golang_code):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(golang_code)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [76]:
ty_code = """
function calculatePrimes(n: number): number[] {
    // Create a boolean array to mark non-prime numbers
    const isPrime: boolean[] = new Array(n + 1).fill(true);
    isPrime[0] = isPrime[1] = false;

    // Use Sieve of Eratosthenes algorithm
    for (let i = 2; i * i <= n; i++) {
        if (isPrime[i]) {
            for (let j = i * i; j <= n; j += i) {
                isPrime[j] = false;
            }
        }
    }

    // Collect prime numbers into result array
    const primes: number[] = [];
    for (let i = 2; i <= n; i++) {
        if (isPrime[i]) {
            primes.push(i);
        }
    }

    return primes;
}

calculatePrimes(1000000);
"""

In [77]:
optimize_gpt(ty_code)

In [78]:
optimize_claude(ty_code)

In [79]:
def stream_gpt(ts_code):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(ts_code), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```golang\n','').replace('```','')

In [80]:
def stream_claude(ts_code):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(ts_code)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```golang\n','').replace('```','')

In [81]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far        

In [84]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Typescript code:", lines=10, value=ty_code)
        cpp = gr.Textbox(label="Golang code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)